# Writing in an existing netcdf file

In [1]:
from netCDF4 import Dataset

In [5]:
data = Dataset('/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d30.1d_curloverf.nc', 'r')

In [6]:
print data.file_format

NETCDF4


In [30]:
out=Dataset('/media/extra/DATA/NATL60/NATL60-CJM165-S/test.nc','w')

In [31]:
for dname, the_dim in data.dimensions.iteritems():
    print dname, len(the_dim)
    out.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)


# Copy variables
for v_name, varin in data.variables.iteritems(): 
    print v_name
    if v_name == 'socurloverf': 
      continue
    outVar = out.createVariable(v_name, varin.datatype, varin.dimensions)
    # Copy variable attributes
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]



x 5422
y 3454
unknown 1
time_counter 1
nav_lon
nav_lat
unknown
time_counter
socurloverf


In [34]:
curlin=data['socurloverf']
curlout=out.createVariable('socurloverf',curlin.datatype,curlin.dimensions)
curlout.setncatts({k: curlin.getncattr(k) for k in curlin.ncattrs()})


In [35]:
curlout[:]=0

In [19]:
for name, dimension in data.dimensions.iteritems():
    out.createDimension(name, len(dimension) if not dimension.isunlimited() else None)

In [20]:
out

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(5422), y(3454), unknown(1), time_counter(0)
    variables(dimensions): 
    groups: 

In [21]:
for name, variable in data.variables.iteritems():
    print name
    # take out the variable you don't want
    if name == 'socurloverf': 
      continue

    x = out.createVariable(name, variable.datatype, variable.dimensions)
    out.variables[x][:] = data.variables[x][:]

nav_lon


KeyError: <type 'netCDF4._netCDF4.Variable'>
float32 nav_lon(y, x)
unlimited dimensions: 
current shape = (3454, 5422)
filling on, default _FillValue of 9.96920996839e+36 used


In [36]:
print data.variables['socurloverf']

<type 'netCDF4._netCDF4.Variable'>
float32 socurloverf(time_counter, y, x)
    units: -
    _FillValue: 0.0
    valid_min: -1000.0
    valid_max: 1000.0
    long_name: Relative_Vorticity (curl)
    short_name: socurl
    iweight: 1
    online_operation: N/A
    axis: TYX
    scale_factor: 1.0
    add_offset: 0.0
    savelog10: 0.0
unlimited dimensions: time_counter
current shape = (1, 3454, 5422)
filling on


In [43]:
curl=data.variables['socurloverf'][:]

In [38]:
filt_curl=data.createVariable("filt_curl","f4",("time_counter","y","x",))

In [39]:
print data.variables['filt_curl']

<type 'netCDF4._netCDF4.Variable'>
float32 filt_curl(time_counter, y, x)
unlimited dimensions: time_counter
current shape = (1, 3454, 5422)
filling on, default _FillValue of 9.96920996839e+36 used



In [40]:
filt_curl[:]=0.

In [41]:
data.close()

In [20]:
test=data.get_variables_by_attributes(short_name="socurl")

In [22]:
test2=data.variables['socurloverf'][:]

In [23]:
test2.shape

(1, 3454, 5422)

In [24]:
test2[:]=0.

In [27]:
data.variables['socurloverf'][:]=test2[:]

RuntimeError: NetCDF: Not a valid ID

In [25]:
data.close()

In [54]:
print time.isunlimited()

True


In [63]:
w_nc_fid = Dataset('/media/extra/DATA/NATL60/NATL60-CJM165-S/test8.nc', 'w', format='NETCDF4')
w_nc_fid.description = "NATL60 data filtered with Lanczos filter with cut-off frequency of 2deg"

time=w_nc_fid.createDimension('time_counter',None)
x=w_nc_fid.createDimension('x', 5422)
y=w_nc_fid.createDimension('y', 3454)
w_nc_var = w_nc_fid.createVariable('truc', 'f8', ('time_counter','y','x'))
w_nc_var.shape
w_nc_var[1,:,:] = 1.
w_nc_fid.close()  # close the new file


(0, 3454, 5422)

In [60]:
curl.shape

(1, 3454, 5422)

In [48]:
curl

masked_array(data =
 [[[-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  ..., 
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]]],
             mask =
 [[[ True  True  True ...,  True  True  True]
  [ True  True  True ...,  True  True  True]
  [ True  True  True ...,  True  True  True]
  ..., 
  [ True  True  True ...,  True  True  True]
  [ True  True  True ...,  True  True  True]
  [ True  True  True ...,  True  True  True]]],
       fill_value = 0.0)

In [51]:
curl[0,100,100]

-0.1615604

In [72]:
filtset = Dataset('/media/extra/DATA/NATL60/NATL60-CJM165-S/test12.nc', 'w', format='NETCDF4')
filtset.description = "NATL60 data filtered with Lanczos filter with cut-off frequency of 2deg"

time=filtset.createDimension('time_counter',None)
x=filtset.createDimension('x', 5422)
y=filtset.createDimension('y', 3454)

lat = filtset.createVariable('nav_lat', 'f8', ('y','x'))
lat.standart_name="latitude"
lat.long_name = "Latitude" 
lat.units = "degrees_north"
lat.nav_model = "grid_U"

lon = filtset.createVariable('nav_lon', 'f8', ('y','x'))
lon.standard_name = "longitude" 
lon.long_name = "Longitude" 
lon.units = "degrees_east" 
lon.nav_model = "grid_U" 
    
time = filtset.createVariable('time_counter', 'f8', ('time_counter'))
time.axis = "T" 
time.standard_name = "time" 
time.long_name = "Time axis" 
time.calendar = "gregorian" 
time.units = "seconds since 1958-01-01 00:00:00" 
time.time_origin = "1958-01-01 00:00:00" 
time.bounds = "time_counter_bounds" 

socurloverf = filtset.createVariable('socurloverf', 'f8', ('time_counter','y','x'),fill_value=0.)
socurloverf.units = "-" 
socurloverf.valid_min = -1000.
socurloverf.valid_max = 1000.
socurloverf.long_name = "Relative_Vorticity (curl)" 
socurloverf.short_name = "socurl" 
socurloverf.iweight = 1 
socurloverf.online_operation = "N/A" 
socurloverf.axis = "TYX" 
socurloverf.scale_factor = 1.
socurloverf.add_offset = 0.
socurloverf.savelog10 = 0. 


socurloverf.shape
socurloverf[0,:,:] = 1.
filtset.close()  # close the new file


In [73]:
filtset = Dataset('/media/extra/DATA/NATL60/NATL60-CJM165-S/test12.nc', 'r')
curl=filtset.variables['socurloverf'][:]
curl


array([[[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ..., 
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]]])

In [74]:
curl.shape

(1, 3454, 5422)

In [75]:
import numpy as npy

In [81]:
npy.arange(0,31)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])